In [1]:
import sys, os
os.path.abspath('..') in sys.path or sys.path.append(os.path.abspath('..'))

In [2]:
import pandas as pd
import petl as etl
from utils.paths import OPEN_DATA, SITE

In [3]:
BRADFORD_DG = SITE / 'observatory/demographics/_data/'
BRADFORD_DG.mkdir(exist_ok=True, parents=True)

In [4]:
census = etl.fromdataframe(pd.read_parquet(OPEN_DATA / 'census.parquet'))

In [5]:
categories = etl.lookup(
    etl.fromcsv(OPEN_DATA / 'census_categories.csv').sort('sortorder'),
    key='variable',
    value='category'
)

In [6]:
def prepare_data(variable, folder):
    folder.mkdir(exist_ok=True, parents=True)
    data = (
        census
        .selecteq('variable', variable)
        .selecteq('measure', 'Percent')
        .cutout('variable', 'measure')
        .recast(variablefield='category')
        .cut('geography_code', 'geography_name', *categories.get(variable))
    )
    data.tocsv(folder / 'by_geography.csv')

prepare_data('Age', BRADFORD_DG / 'age')
prepare_data('Sex', BRADFORD_DG / 'sex')
prepare_data('Ethnic group', BRADFORD_DG / 'ethnic_group')